In [1]:
# (ignore this) imports and utility functions

import re
import json
import toolkit

from textwrap import dedent
from IPython.display import clear_output, display, Markdown


def render_markdown(text):
    if text is None:
        text = ""

    pattern = r"```thinking\n(.*?)\n```"

    def replacement(match):
        content = match.group(1)
        return f"<blockquote>\n{content}\n</blockquote>\n\n"

    text = re.sub(pattern, replacement, text, flags=re.DOTALL)

    text = re.sub(r"```function_spec\b", "```json", text)
    text = re.sub(r"```function_call\b", "```json", text)
    text = re.sub(r"```function_output\b", "```json", text)

    display(Markdown(text))


def render_json(data):
    return render_markdown(f"```json\n{json.dumps(data, indent=2)}\n```\n")


def render_live(stream):
    response = ""
    render_markdown("> Loading...")

    for chunk in stream:
        response += chunk
        clear_output(wait=True)
        render_markdown(response)

    return response

In [2]:
model = "gemma3:27b"

In [3]:
with open("schemas/function.schema.json") as file:
    schema = json.load(file)

In [4]:
await toolkit.code.create_sandbox()

In [5]:
task = dedent("""
    Generate a simple python function that fetches the weather for any place using the wttr.in API.

    The API is simple, just pass the name of the place like so:

        "http://wttr.in/<place>?format=(%l) %C and %t (feels %f) with winds at %w"

    The place can be a place name (Mumbai), an airport code (BOM), or an iconic location (Gateway of India).

    Do not generate examples that use the function. Do not print the result, return it from the function. Do not
    handle errors by printing or ignoring them, raise a custom exception with a user-friendly message instead.
""")

In [6]:
messages = [{"role": "user", "content": task}]

In [7]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

```python
import requests

class WeatherFetchError(Exception):
    """Custom exception for weather fetching errors."""
    pass

def get_weather(place):
    """
    Fetches the weather for a given place using the wttr.in API.

    Args:
        place (str): The name of the place, airport code, or iconic location.

    Returns:
        str: The weather information as a string.

    Raises:
        WeatherFetchError: If the request to wttr.in fails.
    """
    url = f"http://wttr.in/{place}?format=(%l) %C and %t (feels %f) with winds at %w"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response.text
    except requests.exceptions.RequestException as e:
        raise WeatherFetchError(f"Failed to fetch weather for {place}: {e}")
```

In [8]:
function = toolkit.functions.parse_funcs(complete_response)[0]

In [9]:
task = dedent(f"""
    Generate a function specification for the function you just generated. Make sure it is produced in a code block
    with the language set to `function_spec` instead of `json`. Make sure to follow the below schema:

    {json.dumps(schema, indent=2)}
""")

In [10]:
messages.append({"role": "user", "content": task})

In [11]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

```json
{
  "name": "get_weather",
  "description": "Fetches the current weather information for a specified location using the wttr.in API. The location can be a city name, airport code, or iconic landmark.",
  "parameters": {
    "type": "object",
    "properties": {
      "place": {
        "type": "string",
        "description": "The name of the place (e.g., London, JFK, Eiffel Tower) for which to retrieve the weather."
      }
    },
    "required": [
      "place"
    ]
  },
  "responses": [
    {
      "type": "string",
      "description": "A string containing the formatted weather information, as provided by the wttr.in API.  For example:  \"London: Condition: Cloudy, Temperature: 12°C (Feels like 10°C), Wind: 15 km/h from the west\"."
    }
  ],
  "errors": [
    {
      "name": "WeatherFetchError",
      "description": "Raised when there is an error fetching the weather information, such as an invalid location or a network issue."
    }
  ],
  "examples": []
}
```

In [12]:
specification = toolkit.functions.parse_specs(complete_response)[0]

In [13]:
function = await toolkit.code.define_function(specification["name"], function)
toolkit.functions.register(function, specification)

In [14]:
prompt = toolkit.prompts.function_calling(model)
messages.append({"role": "user", "content": prompt})

In [15]:
task = "What's the weather in Pune and Singapore?"

In [16]:
messages.append({"role": "user", "content": task})

In [17]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
Okay, the user wants to know the weather in two locations: Pune and Singapore. I have a function `get_weather` that can fetch weather information for a given place. I can call this function twice, once for each location. These function calls are independent of each other, so I can make them in parallel. I will include two `function_call` blocks in my response, one for Pune and one for Singapore. I will adhere to the guidelines of only calling independent functions in parallel.
</blockquote>



```json
{
	"id": "pune_weather",
	"function": "get_weather",
	"parameters": {
		"place": "Pune"
	}
}
```

```json
{
	"id": "singapore_weather",
	"function": "get_weather",
	"parameters": {
		"place": "Singapore"
	}
}
```

In [18]:
calls = toolkit.functions.parse_calls(complete_response)
outputs = await toolkit.functions.execute_calls(calls)
render_markdown(outputs)

```json
{
    "id": "pune_weather",
    "result": {
        "result": "(Pune) Patchy rain nearby and +24\u00b0C (feels +26\u00b0C) with winds at \u219222km/h"
    }
}
```

```json
{
    "id": "singapore_weather",
    "result": {
        "result": "(Singapore) Partly cloudy and +27\u00b0C (feels +30\u00b0C) with winds at \u21909km/h"
    }
}
```


In [19]:
messages.append({"role": "user", "content": outputs})

In [20]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
I received the weather information for both Pune and Singapore from the `get_weather` function. Now I can present this information to the user in a clear and concise manner. I have followed the guidelines by only using the information received from the function output and presenting it without any assumptions or modifications.
</blockquote>



The weather in Pune is patchy rain with a temperature of 24°C (feels like 26°C) and winds at 22km/h.

The weather in Singapore is partly cloudy with a temperature of 27°C (feels like 30°C) and winds at 9km/h.